미니배치, 이터레이션, 에포크 참조  
https://wikidocs.net/55580

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from torch.utils.data import DataLoader # 데이터로더
from torch.utils.data import Dataset

In [2]:
xy = np.loadtxt('./data/diabetes.csv.gz', delimiter=',', dtype=np.float32)

In [3]:
print(xy)
print(xy.shape)

[[-0.294118   0.487437   0.180328  ... -0.53117   -0.0333333  0.       ]
 [-0.882353  -0.145729   0.0819672 ... -0.766866  -0.666667   1.       ]
 [-0.0588235  0.839196   0.0491803 ... -0.492741  -0.633333   0.       ]
 ...
 [-0.411765   0.21608    0.180328  ... -0.857387  -0.7        1.       ]
 [-0.882353   0.266332  -0.0163934 ... -0.768574  -0.133333   0.       ]
 [-0.882353  -0.0653266  0.147541  ... -0.797609  -0.933333   1.       ]]
(759, 9)


In [21]:
class DiabetesDataset(Dataset):
    def __init__(self, xy_dataset):
        # 커스텀 데이셋 클래스의 생성자를 정의
        # 데이터를 불러와서 torch.tensor로 할당 및 전처리한다.
        self.x_data = torch.from_numpy(xy_dataset[:, 0:-1])
        self.y_data = torch.from_numpy(xy_dataset[:, [-1]])
        print(f'X shape :{self.x_data.shape} | Y shape :{self.y_data.shape} ')
    
    # 매직 메소드 : 함수 이름 앞, 뒤로 underbar 2개를 붙인 메소드
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        # 데이터셋에서 해당 인덱스에 해당하는 샘플 (x, y)를 가져오는 메소드
        return self.x_data[idx], self.y_data[idx]
        

In [37]:
dataset = DiabetesDataset(xy)
# DataLoader 모듈 : 데이터셋과 배치사이즈를 입력받아서 인스턴스를 생성. 학습할 때 미니배치를 구성해준다.
data_loader = DataLoader(dataset, batch_size=100, num_workers=2) 

X shape :torch.Size([759, 8]) | Y shape :torch.Size([759, 1]) 


## 퀴즈 (Easy)  
1) 당뇨병 데이터셋에 대해서 로지스틱 회귀 모델을 구현한다면 첫번째 레이어의 노드 수는 몇개가 되어야 할까요??   
2) data_loader에서 샘플들을 가져온다면 샘플들의 전체 모양은 어떻게 될까요?  

## 퀴즈 (Normal)  
이전의 노트북 파일을 참고해서 LogisticRegressionModel 클래스를 구현하세요.  
1) 생성자 :  
모델의 은닉층 수는 3층이고 은닉층마다 (8, 6, 4) 개의 뉴런을 가집니다.  
마지막 레이어에는 시그모이드 모듈을 이용해 예측값을 계산하도록 만드세요.  
2) forward :   
최종적으로 확률 값을 예측하도록 레이어를 쌓아서 y를 반환하세요.  

In [28]:
class LogisticRegressionModel(nn.Module):
    def __init__(self):
        super(LogisticRegressionModel, self).__init__()
        self.layer1 = nn.Linear(8, 6)
        self.layer2 = nn.Linear(6, 4)
        self.layer3 = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.layer1(x))
        x = self.sigmoid(self.layer2(x))
        x = self.sigmoid(self.layer3(x))
        y_pred = self.sigmoid(x)
        return y_pred

## 퀴즈 (Normal)  
1) 위 모델을 학습시키기 위해서는 어떤 손실함수를 선택해야할까요??  
2) 위의 로지스틱회귀 클래스 모델을 조금 더 메모리 효율적으로 구현하려면 어떻게 해야할까요?  
3) 위의 로지스틱 회귀 모델을 공책에 그려봅시다. 

In [29]:
model = LogisticRegressionModel()
bce_loss = nn.BCELoss(reduction='mean')
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr)

In [ ]:
list1 = [1, 2, 3, 4, ]

In [34]:
def fit(model, optimizer, loss_func, data_loader, epochs):
    for epoch in range(epochs):
        for batch_index, samples in enumerate(data_loader):
            x_train, y_train = samples
            output = model(x_train)
            optimizer.zero_grad()
            L = loss_func(output, y_train) 
            L.backward() 
            optimizer.step()
            print(f'Epoch: {epoch+1:4d}/{epochs} Batch {batch_index+1:4d}/{len(data_loader)} \
                Loss: {L.item():4f}')

In [35]:
fit(model, optimizer, bce_loss, data_loader, 10)

Epoch:    1/10 Batch    1/8                 Loss: 0.655255
Epoch:    1/10 Batch    2/8                 Loss: 0.659631
Epoch:    1/10 Batch    3/8                 Loss: 0.673315
Epoch:    1/10 Batch    4/8                 Loss: 0.664268
Epoch:    1/10 Batch    5/8                 Loss: 0.618826
Epoch:    1/10 Batch    6/8                 Loss: 0.614176
Epoch:    1/10 Batch    7/8                 Loss: 0.645937
Epoch:    1/10 Batch    8/8                 Loss: 0.668745
Epoch:    2/10 Batch    1/8                 Loss: 0.655032
Epoch:    2/10 Batch    2/8                 Loss: 0.659487
Epoch:    2/10 Batch    3/8                 Loss: 0.673360
Epoch:    2/10 Batch    4/8                 Loss: 0.664185
Epoch:    2/10 Batch    5/8                 Loss: 0.618192
Epoch:    2/10 Batch    6/8                 Loss: 0.613476
Epoch:    2/10 Batch    7/8                 Loss: 0.645639
Epoch:    2/10 Batch    8/8                 Loss: 0.668725
Epoch:    3/10 Batch    1/8                 Loss: 0.6548

## Dataloader를 활용한 학습  

## 퀴즈 (Easy)  
위 학습 반복문에서 enumerate를 사용했습니다. 그 이유가 무엇일까요?  